In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', 80)

In [2]:
df = pd.read_csv('Prepared Data/df.csv')

In [3]:
df

,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,SEASON,AVG_MIN_LAST_5,AVG_FGM_LAST_5,AVG_FGA_LAST_5,AVG_FG_PCT_LAST_5,AVG_FG3M_LAST_5,AVG_FG3A_LAST_5,AVG_FG3_PCT_LAST_5,AVG_FTM_LAST_5,AVG_FTA_LAST_5,AVG_FT_PCT_LAST_5,AVG_OREB_LAST_5,AVG_DREB_LAST_5,AVG_REB_LAST_5,AVG_AST_LAST_5,AVG_STL_LAST_5,AVG_BLK_LAST_5,AVG_TO_LAST_5,AVG_PF_LAST_5,AVG_PTS_LAST_5,AVG_PLUS_MINUS_LAST_5,GAMES_PLAYED_LAST_5,AVG_MIN_LAST_10,AVG_FGM_LAST_10,AVG_FGA_LAST_10,AVG_FG_PCT_LAST_10,AVG_FG3M_LAST_10,AVG_FG3A_LAST_10,AVG_FG3_PCT_LAST_10,AVG_FTM_LAST_10,AVG_FTA_LAST_10,AVG_FT_PCT_LAST_10,AVG_OREB_LAST_10,AVG_DREB_LAST_10,AVG_REB_LAST_10,AVG_AST_LAST_10,AVG_STL_LAST_10,AVG_BLK_LAST_10,AVG_TO_LAST_10,AVG_PF_LAST_10,AVG_PTS_LAST_10,AVG_PLUS_MINUS_LAST_10,GAMES_PLAYED_LAST_10,TEAM_AVG_PTS_LAST_5,TEAM_AVG_FG_PCT_LAST_5,TEAM_AVG_FT_PCT_LAST_5,TEAM_AVG_FG3_PCT_LAST_5,TEAM_AVG_AST_LAST_5,TEAM_AVG_REB_LAST_5,TEAM_AVG_TEAM_WINS_LAST_5,TEAM_AVG_PTS_LAST_10,TEAM_AVG_FG_PCT_LAST_10,TEAM_AVG_FT_PCT_LAST_10,TEAM_AVG_FG3_PCT_LAST_10,TEAM_AVG_AST_LAST_10,TEAM_AVG_REB_LAST_10,TEAM_AVG_TEAM_WINS_LAST_10,RATIO_AVG_PTS_LAST_5_TEAM_AVG_PTS_LAST_5,RATIO_AVG_AST_LAST_5_TEAM_AVG_AST_LAST_5,RATIO_AVG_REB_LAST_5_TEAM_AVG_REB_LAST_5,RATIO_AVG_PTS_LAST_10_TEAM_AVG_PTS_LAST_10,RATIO_AVG_AST_LAST_10_TEAM_AVG_AST_LAST_10,RATIO_AVG_REB_LAST_10_TEAM_AVG_REB_LAST_10,AVERAGE SC RATING,PTS,REB,AST
0,1723,Matt Harpring,UTA,2003,5.00,4.666667,8.000000,0.623667,0.00,0.666667,0.00000,2.333333,2.666667,0.889000,1.333333,2.00,3.333333,1.00,1.000000,0.666667,1.666667,3.666667,11.666667,NaN,0.8,5.000000,4.500000,9.000000,0.551000,0.000000,1.500000,0.000000,2.250000,2.500000,0.916750,1.000000,3.000000,4.000000,1.250000,1.250000,0.750000,2.250000,3.000000,11.250000,NaN,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.6,NaN,NaN,NaN,NaN,NaN,NaN,0.3,NaN,NaN,NaN,NaN,NaN,NaN,3.95,14.0,7.0,0.0
1,1905,Andrei Kirilenko,UTA,2003,5.00,2.666667,5.666667,0.457000,0.00,1.333333,0.00000,1.333333,2.000000,0.722333,1.333333,2.00,3.333333,1.00,1.333333,0.333333,1.666667,2.666667,6.666667,NaN,0.8,5.000000,3.600000,8.200000,0.440800,0.000000,1.800000,0.000000,2.000000,2.400000,0.833400,0.800000,3.400000,4.200000,1.400000,1.400000,0.800000,2.600000,2.600000,9.200000,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,0.6,NaN,NaN,NaN,NaN,NaN,NaN,0.3,NaN,NaN,NaN,NaN,NaN,NaN,3.95,16.0,7.0,1.0
2,731,Greg Ostertag,UTA,2003,5.00,2.250000,5.000000,0.426000,0.00,1.000000,0.00000,1.250000,2.000000,0.666750,1.250000,2.75,4.000000,1.00,1.000000,0.250000,1.500000,2.250000,5.750000,NaN,0.8,5.000000,3.166667,7.333333,0.422833,0.000000,1.500000,0.000000,1.833333,2.333333,0.777833,0.833333,3.666667,4.500000,1.333333,1.166667,0.666667,2.333333,2.333333,8.166667,NaN,0.6,NaN,NaN,NaN,NaN,NaN,NaN,0.6,NaN,NaN,NaN,NaN,NaN,NaN,0.3,NaN,NaN,NaN,NaN,NaN,NaN,3.95,6.0,9.0,1.0
3,2306,Carlos Arroyo,UTA,2003,5.00,1.500000,4.000000,0.476000,0.00,1.000000,0.00000,0.750000,1.250000,0.500000,0.500000,2.50,3.000000,1.75,1.000000,0.250000,1.500000,1.250000,3.750000,NaN,1.0,5.000000,2.857143,6.428571,0.505286,0.000000,1.285714,0.000000,1.571429,2.000000,0.666714,0.714286,3.142857,3.857143,1.571429,1.000000,0.571429,2.000000,2.285714,7.285714,NaN,0.7,NaN,NaN,NaN,NaN,NaN,NaN,0.6,NaN,NaN,NaN,NaN,NaN,NaN,0.3,NaN,NaN,NaN,NaN,NaN,NaN,3.95,18.0,0.0,13.0
4,2052,DeShawn Stevenson,UTA,2003,5.00,1.600000,4.000000,0.480800,0.00,0.800000,0.00000,0.600000,1.000000,0.400000,0.400000,2.20,2.600000,1.60,1.000000,0.200000,1.200000,1.000000,3.800000,NaN,1.0,5.000000,2.750000,6.125000,0.504625,0.000000,1.125000,0.000000,1.375000,1.750000,0.583375,0.625000,2.875000,3.500000,1.500000,1.000000,0.500000,1.750000,2.000000,6.875000,NaN,0.8,NaN,NaN,NaN,NaN,NaN,NaN,0.6,NaN,NaN,NaN,NaN,NaN,NaN,0.3,NaN,NaN,NaN,NaN,NaN,NaN,3.95,6.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277978,1626195,W

In [4]:
df = df.fillna(0)

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = df.drop(labels=['PTS', 'REB', 'AST'], axis=1)

In [7]:
X_num = X.select_dtypes(np.number)
X_cat = X.select_dtypes(object)
X_cat = pd.get_dummies(X_cat)

In [8]:
X = pd.concat([X_cat, X_num], axis=1) 

In [9]:
X.shape

(277983, 1159)

In [10]:
y = df[['PTS', 'REB', 'AST']]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

In [12]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = StandardScaler()

X_train_norm = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

X_test_norm = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

In [13]:
from sklearn.linear_model import RidgeCV, MultiTaskLasso, LinearRegression, ElasticNet, LassoCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.multioutput import RegressorChain, MultiOutputRegressor

from sklearn.model_selection import cross_val_score, RepeatedKFold

In [14]:
lr = LinearRegression()
lasso = ElasticNet() # zeza me lasso, proveriti koji model da se koristi za multiOutpRegress
ridge = RidgeCV()
dtr = DecisionTreeRegressor()
rfr = RandomForestRegressor()
gbr = GradientBoostingRegressor()
mlp = MLPRegressor()

In [15]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [27]:
# cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)

# linear_models = [lr, ridge, mlp, dtr,rfr,gbr]
linear_models = [lr]
for model in linear_models:
    wrapper = MultiOutputRegressor(model)
    wrapper.fit(X_train_norm, y_train)
    y_hat = wrapper.predict(X_test_norm)
    print(f' MAE for {model} is: {round(mean_absolute_error(y_test, y_hat),2)}')
    print()
    print(f' MSE for {model} is: {round(mean_squared_error(y_test, y_hat),2)}')
    print()
    print(' --------------------------')
    print()

 MAE for LinearRegression() is: 324774168.47

 MSE for LinearRegression() is: 1.3920867504183708e+21

 --------------------------

